### Tweet Preprocessing Using ekphrasis
ekphrasis Link: https://github.com/cbaziotis/ekphrasis

ekphrasis offers the following functionality:

1. Social Tokenizer. A text tokenizer geared towards social networks (Facebook, Twitter...), which understands complex emoticons, emojis and other unstructured expressions like dates, times and more.

2. Word Segmentation. You can split a long string to its constituent words. Suitable for hashtag segmentation.

3. Spell Correction. You can replace a misspelled word, with the most probable candidate word.

4. Customization. Taylor the word-segmentation, spell-correction and term identification, to suit your needs.

Word Segmentation and Spell Correction mechanisms, operate on top of word statistics, collected from a given corpus. We provide word statistics from 2 big corpora (from Wikipedia and Twitter), but you can also generate word statistics from your own corpus. You may need to do that if you are working with domain-specific texts, like biomedical documents. For example a word describing a technique or a chemical compound may be treated as a misspelled word, using the word statistics from a general purposed corpus.

ekphrasis tokenizes the text based on a list of regular expressions. You can easily enable ekphrasis to identify new entities, by simply adding a new entry to the dictionary of regular expressions (ekphrasis/regexes/expressions.txt).

5. Pre-Processing Pipeline. You can combine all the above steps in an easy way, in order to prepare the text files in your dataset for some kind of analysis or for machine learning. In addition, to the aforementioned actions, you can perform text normalization, word annotation (labeling) and more.


### Installation the library

In [16]:
# pip install ekphrasis

### Import Required Libraries

In [1]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

### Define a Text Pre-Processing pipeline

You can easily define a preprocessing pipeline, by using the TextPreProcessor.

In [2]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

C:\Users\User\anaconda3\lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams C:\Users\User\.ekphrasis\stats\twitter\counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams C:\Users\User\.ekphrasis\stats\twitter\counts_2grams.txt
Reading twitter - 1grams ...


C:\Users\User\anaconda3\lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


### Test Some tweets

Notes:
1. elongated words are automatically normalized.
2. Spell correction affects performance.


In [5]:
sentences = [
    "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
    "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
    "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/."
]

In [7]:
for s in sentences:
    print(" ".join(text_processor.pre_process_doc(s)))
    print()

<allcaps> cant wait </allcaps> for the new season of <hashtag> twin peaks </hashtag> ＼(^o^)／ ! <repeated> <hashtag> david lynch </hashtag> <hashtag> tv series </hashtag> <happy>

i saw the new <hashtag> john doe </hashtag> movie and it sucks <elongated> ! <repeated> <allcaps> waisted </allcaps> <money> . <repeated> <hashtag> bad movies </hashtag> <annoyed>

<user> : can not wait for the <date> <hashtag> sentiment </hashtag> talks ! <allcaps> yay <elongated> </allcaps> ! <repeated> <laugh> <url>



### Word Segmentation

The word segmentation implementation uses the Viterbi algorithm and is based on CH14 from the book Beautiful Data (Segaran and Hammerbacher, 2009). The implementation requires word statistics in order to identify and separating the words in a string. You can use the word statistics from one of the 2 provided corpora, or from your own corpus.

Example: In order to perform word segmentation, first you have to instantiate a segmenter with a given corpus, and then just use the segment() method.

In [9]:
from ekphrasis.classes.segmenter import Segmenter
seg = Segmenter(corpus="english") 
print(seg.segment("smallandinsignificant"))

Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams C:\Users\User\.ekphrasis\stats\english\counts_1grams.txt
Reading english - 2grams ...
generating cache file for faster loading...
reading ngrams C:\Users\User\.ekphrasis\stats\english\counts_2grams.txt
small and insignificant


#### You can test the output using statistics from the different corpora:

In [10]:
from ekphrasis.classes.segmenter import Segmenter

# segmenter using the word statistics from english Wikipedia
seg_eng = Segmenter(corpus="english") 

# segmenter using the word statistics from Twitter
seg_tw = Segmenter(corpus="twitter")

words = ["exponentialbackoff", "gamedev", "retrogaming", "thewatercooler", "panpsychism"]
for w in words:
    print(w)
    print("(eng):", seg_eng.segment(w))
    print("(tw):", seg_tw.segment(w))
    print()

Reading english - 1grams ...
Reading english - 2grams ...
Reading twitter - 1grams ...
Reading twitter - 2grams ...
exponentialbackoff
(eng): exponential backoff
(tw): exponential back off

gamedev
(eng): gamedev
(tw): game dev

retrogaming
(eng): retrogaming
(tw): retro gaming

thewatercooler
(eng): the water cooler
(tw): the watercooler

panpsychism
(eng): panpsychism
(tw): pan psych is m



#### Finally, if the word is camelCased or PascalCased, then the algorithm splits the words based on the case of the characters.

In [11]:
from ekphrasis.classes.segmenter import Segmenter
seg = Segmenter() 
print(seg.segment("camelCased"))
print(seg.segment("PascalCased"))

Reading english - 1grams ...
Reading english - 2grams ...
camel cased
pascal cased


### Spell Correction

The Spell Corrector is based on Peter Norvig's spell-corrector. Just like the segmentation algorithm, it utilizes word statistics in order to find the most probable candidate. Besides the provided statistics, you can use your own.

##### Example:
You can perform the spell correction, just like the word segmentation. First you have to instantiate a SpellCorrector object, that uses the statistics from the corpus of your choice and then use on of the available methods.

In [12]:
from ekphrasis.classes.spellcorrect import SpellCorrector
sp = SpellCorrector(corpus="english") 
print(sp.correct("korrect"))

Reading english - 1grams ...
correct


### Social Tokenizer

The difficulty in tokenization is to avoid splitting expressions or words that should be kept intact (as one token). This is more important in texts from social networks, with "creative" writing and expressions like emoticons, hashtags and so on. Although there are some tokenizers geared towards Twitter [1],[2], that recognize the Twitter markup and some basic sentiment expressions or simple emoticons, this tokenizer is able to identify almost all emoticons, emojis and many complex expressions.

Especially for tasks such as sentiment analysis, there are many expressions that play a decisive role in identifying the sentiment expressed in text. Expressions like these are:
1. Censored words, such as f**k, s**t.
2. Words with emphasis, such as a *great* time, I don't *think* I ....
3. Emoticons, such as >:(, :)), \o/.
4. Dash-separated words, such as over-consumption, anti-american, mind-blowing.

Moreover, ekphrasis can identify information-bearing expressions. Depending on the task, you may want to keep preserve / extract them as one token (IR) and then normalize them since this information may be irrelevant for the task (sentiment analysis). Expressions like these are:

1. Dates, such as Feb 18th, December 2, 2016, December 2-2016, 10/17/94, 3 December 2016, April 25, 1995, 11.15.16, November 24th 2016, January 21st.
2. Times, such as 5:45pm, 11:36 AM, 2:45 pm, 5:30.
3. Currencies, such as $220M, $2B, $65.000, €10, $50K.
4. Phone numbers.
5. URLs, such as http://www.cs.unipi.gr, https://t.co/Wfw5Z1iSEt.

In [15]:
import nltk
from ekphrasis.classes.tokenizer import SocialTokenizer


def wsp_tokenizer(text):
    return text.split(" ")

puncttok = nltk.WordPunctTokenizer().tokenize

social_tokenizer = SocialTokenizer(lowercase=False).tokenize

sents = [
    "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／ yaaaay!!! #davidlynch #tvseries :)))",
    "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies >3:/",
    "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! >:-D http://sentimentsymposium.com/.",
]

for s in sents:
    print()
    print("ORG: ", s)  # original sentence
    print()
    print("WSP : ", wsp_tokenizer(s))  # whitespace tokenizer
    print()
    print("WPU : ", puncttok(s))  # WordPunct tokenizer
    print()
    print("SC : ", social_tokenizer(s))  # social tokenizer
    print('=====================')


ORG:  CANT WAIT for the new season of #TwinPeaks ＼(^o^)／ yaaaay!!! #davidlynch #tvseries :)))

WSP :  ['CANT', 'WAIT', 'for', 'the', 'new', 'season', 'of', '#TwinPeaks', '＼(^o^)／', 'yaaaay!!!', '#davidlynch', '#tvseries', ':)))']

WPU :  ['CANT', 'WAIT', 'for', 'the', 'new', 'season', 'of', '#', 'TwinPeaks', '＼(^', 'o', '^)／', 'yaaaay', '!!!', '#', 'davidlynch', '#', 'tvseries', ':)))']

SC :  ['CANT', 'WAIT', 'for', 'the', 'new', 'season', 'of', '#TwinPeaks', '＼(^o^)／', 'yaaaay', '!', '!', '!', '#davidlynch', '#tvseries', ':)))']

ORG:  I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies >3:/

WSP :  ['I', 'saw', 'the', 'new', '#johndoe', 'movie', 'and', 'it', 'suuuuucks!!!', 'WAISTED', '$10...', '#badmovies', '>3:/']

WPU :  ['I', 'saw', 'the', 'new', '#', 'johndoe', 'movie', 'and', 'it', 'suuuuucks', '!!!', 'WAISTED', '$', '10', '...', '#', 'badmovies', '>', '3', ':/']

SC :  ['I', 'saw', 'the', 'new', '#johndoe', 'movie', 'and', 'it', 'suuuuucks', '!', '!', 

### Citation

@InProceedings{baziotis-pelekis-doulkeridis:2017:SemEval2,
  author    = {Baziotis, Christos  and  Pelekis, Nikos  and  Doulkeridis, Christos},
  title     = {DataStories at SemEval-2017 Task 4: Deep LSTM with Attention for Message-level and Topic-based Sentiment Analysis},
  booktitle = {Proceedings of the 11th International Workshop on Semantic Evaluation (SemEval-2017)},
  month     = {August},
  year      = {2017},
  address   = {Vancouver, Canada},
  publisher = {Association for Computational Linguistics},
  pages     = {747--754}
}

### References

[1] K. Gimpel et al., “Part-of-speech tagging for twitter: Annotation, features, and experiments,” in Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies: short papers-Volume 2, 2011, pp. 42–47.

[2] C. Potts, “Sentiment Symposium Tutorial: Tokenizing,” Sentiment Symposium Tutorial, 2011. [Online]. Available: http://sentiment.christopherpotts.net/tokenizing.html.